In [1]:
with open("out.tf", "r") as f:
    data = f.readlines()

In [2]:
EVENT, TIME, FROM_NODE, TO_NODE, PACKET_TYPE, PACKET_SIZE, FLAGS, FID, SOURCE_ADDRESS, DST_ADDRESS, SEQ_NUM, PKT_ID = [i for i in range(12)]

In [3]:
def filterPackets(data, condition):
    result = []
    for line in data:
        items = line.split()
        conditionSatisfied = True
        for key, value in condition.items():
            if items[key] != value:
                conditionSatisfied = False
                break
        if conditionSatisfied:
            result.append(items)
    return result

In [4]:
def getTotalPacketsSize(connection):
    return sum([int(packets[PACKET_SIZE]) for packets in connection])

In [7]:
ftp0Condition = {TO_NODE: '5', EVENT: 'r'}
ftp1Condition = {TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'tcp'}
ftp2Condition = {TO_NODE: '7', EVENT: 'r'}
udpCondition = {TO_NODE: '6', EVENT: 'r', PACKET_TYPE: 'cbr'}
ftp0PacketSum = getTotalPacketsSize(filterPackets(data, ftp0Condition))
ftp1PacketSum = getTotalPacketsSize(filterPackets(data, ftp1Condition))
ftp2PacketSum = getTotalPacketsSize(filterPackets(data, ftp2Condition))
udpPacketSum = getTotalPacketsSize(filterPackets(data, udpCondition))
print("Average Throughput of TCP0:", ftp0PacketSum / 10)
print("Average Throughput of TCP1:", ftp1PacketSum / 10)
print("Average Throughput of TCP2:", ftp2PacketSum / 10)
print("Average Throughput of UDP:", udpPacketSum / 1)

Average Throughput of TCP0: 106292.0
Average Throughput of TCP1: 37132.0
Average Throughput of TCP2: 34116.0
Average Throughput of UDP: 128000.0


In [16]:
def getAverageDelay(data, startCondition, endCondition):
    firstPackets = filterPackets(data, startCondition)
    startTime = {}
    for item in firstPackets:
        startTime[item[PKT_ID]] = float(item[TIME])
    lastPackets = filterPackets(data, endCondition)
    totalDelay = 0
    for item in lastPackets:
        totalDelay += float(item[TIME]) - startTime[item[PKT_ID]]
    return totalDelay / len(lastPackets)

Average Delay of TCP0: 0.21389742521994065


In [17]:
print("Average Delay of TCP0:", getAverageDelay(data, {FROM_NODE: '0', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '5', EVENT: 'r'}))
print("Average Delay of TCP1:", getAverageDelay(data, {FROM_NODE: '0', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '5', EVENT: 'r'}))
print("Average Delay of TCP2:", getAverageDelay(data, {FROM_NODE: '0', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '5', EVENT: 'r'}))
print("Average Delay of UDP:", getAverageDelay(data, {FROM_NODE: '0', TO_NODE: '3', EVENT: '+'}, {FROM_NODE: '4', TO_NODE: '5', EVENT: 'r'}))